# Dask Array (1)

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Dask"
* https://docs.dask.org/en/latest/array.html
* https://docs.dask.org/en/stable/array-chunks.html
* https://en.wikipedia.org/wiki/Taxicab_geometry
* https://docs.h5py.org/en/stable/

## Задачи для совместного разбора

1. Создайте массив размерностью 1000 на 300000, заполненный числами из стандартного нормального распределения. Исследуйте основные характеристики полученного массива.

2. Посчитайте сумму квадратов элементов массива, созданного в задаче 1. Создайте массив `np.array` такого же размера и сравните скорость решения задачи с использование `da.array` и `np.array`

## Лабораторная работа 7

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy`, `pandas` и `dask`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy`, структур `pandas` или структур `dask` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

В ходе выполнения все операции вычислений (расчет средних значений, расчет косинусной близости и т.д.) проводятся над `dask.array` и средствами пакета `dask`, если в задании не сказано обратного. Переход от `dask.array` к `numpy.array` или `pd.DataFrame` возможен исключительно для демонстрации результата в конце решения задачи. Если в задаче используются результаты выполнения предыдущих задач, то подразумевается, что вы используете результаты в виде `dask.array` (то есть то, что было получено до вызова `compute`, а не после).

In [1]:
import dask.array as da
import dask
import h5py
import numpy as np
import pandas as pd
from multiprocessing.pool import ThreadPool

<p class="task" id="1"></p>

1\. Считайте датасет `embeddings` из файла `recipe_embeddings.h5` в виде `dask.array`. Выведите на экран основную информацию о массиве: размер, количество векторов $M$ (количество строк в массиве), размерность каждого вектора $N$ (количество столбцов в массиве), тип, количество и размер сегментов. 

In [2]:
file = h5py.File('recipe_embeddings.h5')
print(file['embeddings'])
array = da.from_array(file['embeddings'])
array

<HDF5 dataset "embeddings": shape (1200000, 312), type "<f4">


dask.array<array, shape=(1200000, 312), dtype=float32, chunksize=(100000, 312), chunktype=numpy.ndarray>

<p class="task" id="2"></p>

2\. Посчитайте и выведите на экран среднее значение всех элементов массива. Исследуйте, как влияет значение аргумента `chunks` при создании `dask.array` на скорость выполнения операции поиска среднего. 

Пусть $M$ - количество строк в массиве, $N$ - количество столбцов в массиве, `chunks=(r,c)`. Сравните несколько вариантов:
* $r=M$, $с \ll N$ , 
* $r \ll M$, $c=N$ 
* $r = M$, $c = N$ 
* значения $r, c$ по умолчанию.

Выберите наиболее оптимальные значения $r$ и  $c$ в смысле скорости вычислений и далее продолжайте работу с ними.

In [3]:
%%time
#С дефолтными значениями
m = array.mean(axis=0)
m.compute()

Wall time: 502 ms


array([ 9.75613308e-04, -3.39259915e-02,  4.66551557e-02, -8.74476507e-02,
       -9.71177034e-03,  5.51532954e-03,  2.71454044e-02,  3.75370905e-02,
       -2.35138610e-02, -1.10451411e-02,  2.61558332e-02,  8.86423327e-03,
        1.82165187e-02,  5.94256371e-02,  2.72058621e-02, -6.64571533e-03,
        3.04658655e-02,  1.03974920e-02,  1.93780344e-02,  1.43449485e-01,
       -2.46166624e-03, -5.94153255e-03, -3.18063907e-02, -4.10590284e-02,
        8.61882269e-02,  2.80386396e-02, -2.43214611e-02, -4.59721498e-03,
        1.09600713e-02,  3.78382765e-02, -1.06729204e-02, -1.57453716e-02,
        1.08159299e-03, -2.18519215e-02,  7.02830730e-03,  5.17747067e-02,
       -9.01941909e-04, -3.80985737e-02, -7.50487596e-02,  1.78839285e-02,
       -6.01360276e-02,  1.52848035e-01,  7.50525817e-02, -3.78861800e-02,
       -2.17200909e-02,  3.46800871e-03,  3.78258862e-02, -4.40570116e-02,
        4.40716483e-02, -3.56613398e-02,  8.02602526e-03, -3.51645462e-02,
       -4.23886068e-02,  

In [4]:
%%time
#Первый вариант
array1 = da.from_array(file['embeddings'], chunks=(1200000, 30))
m1 = array1.mean(axis=0)
m1.compute()

Wall time: 4.18 s


array([ 9.75604577e-04, -3.39262448e-02,  4.66538034e-02, -8.74485672e-02,
       -9.71151888e-03,  5.51547715e-03,  2.71460190e-02,  3.75387222e-02,
       -2.35141497e-02, -1.10451020e-02,  2.61556506e-02,  8.86390824e-03,
        1.82161387e-02,  5.94242699e-02,  2.72051897e-02, -6.64538704e-03,
        3.04663479e-02,  1.03973076e-02,  1.93782579e-02,  1.43450215e-01,
       -2.46160291e-03, -5.94158005e-03, -3.18066292e-02, -4.10596021e-02,
        8.61877501e-02,  2.80382913e-02, -2.43208390e-02, -4.59714327e-03,
        1.09600378e-02,  3.78392376e-02, -1.06730014e-02, -1.57452952e-02,
        1.08159077e-03, -2.18521748e-02,  7.02842185e-03,  5.17745838e-02,
       -9.01937892e-04, -3.80997583e-02, -7.50498995e-02,  1.78837068e-02,
       -6.01354428e-02,  1.52847350e-01,  7.50514269e-02, -3.78870741e-02,
       -2.17200741e-02,  3.46807414e-03,  3.78268287e-02, -4.40578088e-02,
        4.40724567e-02, -3.56607772e-02,  8.02617799e-03, -3.51648331e-02,
       -4.23883013e-02,  

In [5]:
%%time
#Второй вариант
array2 = da.from_array(file['embeddings'], chunks=(300000, 312))
print(type(array2))
m2 = array2.mean(axis=0)
m2.compute()

<class 'dask.array.core.Array'>
Wall time: 513 ms


array([ 9.75608302e-04, -3.39256376e-02,  4.66548763e-02, -8.74479488e-02,
       -9.71178524e-03,  5.51532069e-03,  2.71453001e-02,  3.75370979e-02,
       -2.35139076e-02, -1.10451868e-02,  2.61558965e-02,  8.86422396e-03,
        1.82164349e-02,  5.94252013e-02,  2.72057820e-02, -6.64574280e-03,
        3.04657854e-02,  1.03975348e-02,  1.93780847e-02,  1.43450990e-01,
       -2.46164715e-03, -5.94153861e-03, -3.18065137e-02, -4.10590433e-02,
        8.61890316e-02,  2.80384570e-02, -2.43213661e-02, -4.59720567e-03,
        1.09601198e-02,  3.78382243e-02, -1.06729055e-02, -1.57452896e-02,
        1.08159904e-03, -2.18517408e-02,  7.02825747e-03,  5.17750122e-02,
       -9.01948253e-04, -3.80986929e-02, -7.50487596e-02,  1.78839955e-02,
       -6.01364858e-02,  1.52848050e-01,  7.50515908e-02, -3.78861055e-02,
       -2.17200667e-02,  3.46802734e-03,  3.78254913e-02, -4.40571494e-02,
        4.40715291e-02, -3.56614217e-02,  8.02607555e-03, -3.51643488e-02,
       -4.23886478e-02,  

In [6]:
%%time
#Третий вариант
array3 = da.from_array(file['embeddings'], chunks=(1200000, 312))
m3 = array3.mean(axis=0)
m3.compute()

Wall time: 617 ms


array([ 9.75604577e-04, -3.39262448e-02,  4.66538034e-02, -8.74485672e-02,
       -9.71151888e-03,  5.51547715e-03,  2.71460190e-02,  3.75387222e-02,
       -2.35141497e-02, -1.10451020e-02,  2.61556506e-02,  8.86390824e-03,
        1.82161387e-02,  5.94242699e-02,  2.72051897e-02, -6.64538704e-03,
        3.04663479e-02,  1.03973076e-02,  1.93782579e-02,  1.43450215e-01,
       -2.46160291e-03, -5.94158005e-03, -3.18066292e-02, -4.10596021e-02,
        8.61877501e-02,  2.80382913e-02, -2.43208390e-02, -4.59714327e-03,
        1.09600378e-02,  3.78392376e-02, -1.06730014e-02, -1.57452952e-02,
        1.08159077e-03, -2.18521748e-02,  7.02842185e-03,  5.17745838e-02,
       -9.01937892e-04, -3.80997583e-02, -7.50498995e-02,  1.78837068e-02,
       -6.01354428e-02,  1.52847350e-01,  7.50514269e-02, -3.78870741e-02,
       -2.17200741e-02,  3.46807414e-03,  3.78268287e-02, -4.40578088e-02,
        4.40724567e-02, -3.56607772e-02,  8.02617799e-03, -3.51648331e-02,
       -4.23883013e-02,  

По результатам тестирования видно, что дефолтные значения при создании массива оказались самыми эффективными, но, посколькку нужно выбрать из предложенных, дальше будем работать со вторым вариантом (array2 r<<M c=N)

<p class="task" id="3"></p>

3\. Опишите пространство, в котором расположены эмбеддинги, посчитав минимальное и максимальное значение для каждой из координат. Сведите результаты в таблицу `pd.DataFrame`, состоящую из двух строк и 312 столбцов. Задайте индексы строк "min" и "max". Названия столбцов сделайте вида $e_i$. Выведите полученную таблицу на экран.

Решите задачу двумя способами. В первом варианте сделайте два вызова метода `compute` для расчета каждого из векторов максимальных и минимальных значений. Во втором варианте сделайте один вызов функции `dask.compute` для одновременного расчета двух векторов. Сравните время выполнения двух решений.

In [7]:
q1 = array2.max(axis=0)
q2 = array2.min(axis=0)
print(q1.compute())
print(q2.compute())

[ 0.13503821  0.07612539  0.1578539   0.03098669  0.10119153  0.1117736
  0.1474969   0.17382096  0.09980789  0.11557347  0.16072085  0.13614471
  0.12693979  0.19202124  0.15039271  0.11139409  0.15655707  0.11023002
  0.10263447  0.2901863   0.12586455  0.11356723  0.08866968  0.08353768
  0.2120125   0.14561214  0.1162458   0.13361782  0.13729496  0.17057817
  0.10226841  0.10846934  0.12675716  0.12531523  0.1093971   0.17144266
  0.11409133  0.10032833  0.1120521   0.13663255  0.07725815  0.24485739
  0.22898078  0.06262603  0.07915583  0.13622959  0.16562136  0.10411018
  0.17232957  0.10518377  0.12112924  0.11170167  0.07890626  0.14746335
  0.07686806  0.1650769   0.16921823  0.14506307  0.15775387  0.14433634
  0.12886478  0.17552072  0.1157431   0.17182784  0.11604308  0.2436741
  0.20468575  0.09141792  0.14633343  0.17374453  0.2212709   0.12169629
  0.09666786  0.01166006  0.1269079   0.10479862  0.11348008  0.10815039
  0.17101324  0.15925443  0.12972395  0.14320208  0.0

In [8]:
df = pd.DataFrame(data=[q2.compute(), q1.compute()], columns=[f'e{i}' for i in range(312)])

In [9]:
df.index = ['min', 'max']

In [10]:
df

,e0,e1,e2,e3,e4,e5,e6,e7,e8,e9,...,e302,e303,e304,e305,e306,e307,e308,e309,e310,e311
min,-0.132803,-0.149056,-0.094468,-0.191697,-0.114229,-0.114341,-0.096039,-0.115178,-0.157275,-0.116715,...,-0.103254,-0.122285,-0.149789,-0.127703,-0.094802,-0.119690,-0.141425,-0.123732,-0.081543,-0.227348
max,0.135038,0.076125,0.157854,0.030987,0.101192,0.111774,0.147497,0.173821,0.099808,0.115573,...,0.119518,0.197589,0.113135,0.136490,0.162921,0.099021,0.086653,0.158176,0.166968,0.048967


In [11]:
min_val, max_val = dask.compute(array2.min(axis=0), array2.max(axis=0))
print(len(min_val))
print(len(max_val))

312
312


In [12]:
dask.compute(array2.min(axis=0), array2.max(axis=0))

(array([-0.13280304, -0.14905587, -0.09446791, -0.1916968 , -0.11422908,
        -0.11434078, -0.09603858, -0.11517793, -0.15727527, -0.1167149 ,
        -0.11471461, -0.15019277, -0.0796339 , -0.0945342 , -0.09285595,
        -0.13958164, -0.09811686, -0.0975826 , -0.10431358, -0.05787669,
        -0.13606346, -0.12994963, -0.14080842, -0.16251615, -0.07816251,
        -0.08777131, -0.15673235, -0.1112382 , -0.11480374, -0.08176571,
        -0.11284319, -0.16102342, -0.11257177, -0.15000317, -0.12035861,
        -0.0584373 , -0.11112811, -0.15351729, -0.22966738, -0.14327545,
        -0.1974432 ,  0.03151742, -0.09751797, -0.129823  , -0.10871614,
        -0.13429932, -0.07607475, -0.14919333, -0.07580777, -0.16950257,
        -0.11487022, -0.15702714, -0.1671625 , -0.10305755, -0.21833259,
        -0.08830633, -0.05387363, -0.09623961, -0.12235058, -0.09821943,
        -0.04651169, -0.08259498, -0.11942946, -0.0804956 , -0.11324544,
        -0.0824606 , -0.14259079, -0.11750264, -0.1

In [13]:
df1 = pd.DataFrame(dask.compute(array2.min(axis=0), array2.max(axis=0)), columns=[f'e{i}' for i in range(312)])

In [14]:
df1.index = ['min', 'max']

In [15]:
df1

,e0,e1,e2,e3,e4,e5,e6,e7,e8,e9,...,e302,e303,e304,e305,e306,e307,e308,e309,e310,e311
min,-0.132803,-0.149056,-0.094468,-0.191697,-0.114229,-0.114341,-0.096039,-0.115178,-0.157275,-0.116715,...,-0.103254,-0.122285,-0.149789,-0.127703,-0.094802,-0.119690,-0.141425,-0.123732,-0.081543,-0.227348
max,0.135038,0.076125,0.157854,0.030987,0.101192,0.111774,0.147497,0.173821,0.099808,0.115573,...,0.119518,0.197589,0.113135,0.136490,0.162921,0.099021,0.086653,0.158176,0.166968,0.048967


<p class="task" id="4"></p>

4\. Датасет `embeddings` представляет собой набор 312-мерных векторов $x_i, i=0, 1, ... M-1$ Найдите вектор $x \ne x_{256}$ из набора данных, ближайший к вектору $x_{256}$ в смысле метрики $L_1$. Выведите на экран первые 10 координат вектора $x$.

$$d_1(\textbf{x},\textbf{y})=\sum_{k=1}^{n}{|x_i - y_i|}, \textbf{x}, \textbf{y} \in \mathbb{R}^n$$

In [16]:
array2[256].compute()[:10]

array([-0.01716741, -0.05312856,  0.07754256, -0.12590791, -0.01951944,
       -0.01012862,  0.00996168,  0.00411218,  0.01872775, -0.02361023],
      dtype=float32)

In [17]:
%%time
x256 = array2[256]

max_val = da.max(array2)
array2[256] = max_val

distances = da.sum(da.abs(array2 - x256), axis=1)
nearest_index = da.argmin(distances)
x_nearest = array2[nearest_index]

print(len(x_nearest))
print(x_nearest[:10].compute())
print(nearest_index.compute())

312
[ 0.0331987  -0.03648246  0.06629294 -0.0850755  -0.04708353  0.00130241
  0.00259956  0.01916818 -0.00985817 -0.04410348]
1132465
Wall time: 2.05 s


In [18]:
array2 = da.from_array(file['embeddings'], chunks=(300000, 312))

<p class="task" id="5"></p>

5\. Рецепты разбиты на 4 группы. Загрузите маску для разбиения на группы из датасета `mask` из файла `recipe_embeddings.h5` в виде `dask.array`. Для каждой группы посчитайте и выведите на экран максимальное значение нормы $\ell_1$ векторов рецептов, принадлежащих к этой группе. 

Подсказка: закодируйте маску принадлежности к группе при помощи метода кодирования one-hot encoding и воспользуйтесь механизмом распространения.

$$\ell_1: ||\textbf{x}||_1=\sum_{k=1}^{n}{|x_k|}, \textbf{x} \in \mathbb{R}^n$$

<p class="task" id="6"></p>

6\. Работая с исходным файлом в формате `hdf`, реализуйте алгоритм подсчета среднего вектора датасета в блочной форме.

Блочный алгоритм вычислений состоит из двух частей:
1. Загрузка фрагмента за фрагментом данных и проведение вычислений над этим фрагментом
2. Агрегация результатов вычислений на различных фрагментах для получения результата на уровне всего набора данных

Важно: при работе с `hdf` в память загружаются не все элементы, а только те, которые запрашиваются в данный момент. При работе с `hdf` вы можете работать с массивами `numpy.array`. Для итерации по сегментам файла допускается использование циклов.

In [19]:
%%time
f = h5py.File('recipe_embeddings.h5')
ds = f['embeddings']
num_vectors = len(ds) # кол-во векторов
shape_vectors = len(ds[0])
sum_vector = np.zeros(shape_vectors, dtype=np.float32) # Массив для хранения суммы векторов
block_size = 1000 # размер одного блока
num_blocks = num_vectors // block_size # количество блоков

# Проходим по всем блокам и суммируем векторы в каждом блоке
for i in range(num_blocks):
    start = i * block_size
    end = min(start + block_size, num_vectors)
    block_vectors = ds[start:end]
    sum_vector += np.sum(block_vectors, axis=0)
mean_vector = sum_vector / num_vectors
    
f.close()
mean_vector[:10]

Wall time: 750 ms


array([ 0.00097561, -0.033926  ,  0.04665515, -0.08744798, -0.00971177,
        0.00551533,  0.02714543,  0.03753719, -0.02351383, -0.01104514])

<p class="task" id="7"></p>

7\. Решите задачу 6, распараллелив вычисления при помощи `ThreadPool`. Сравните время и результаты решения работы вашего алгоритма с реализацией поиска среднего вектора из `dask`. 

In [20]:
%%time
block_size = 1000

def process_block(block_index):
    with h5py.File('recipe_embeddings.h5', 'r') as f:
        dataset = f['embeddings']
        num_vectors = len(dataset) # кол-во векторов
        shape_vectors = len(dataset[0])
        sum_vector = np.zeros(shape_vectors, dtype=np.float32) # Массив для хранения суммы векторов
        num_blocks = num_vectors // block_size # количество блоков
        start_index = block_index * block_size
        end_index = min(start_index + block_size, dataset.shape[0])
        block_vectors = dataset[start_index:end_index]
        return np.sum(block_vectors, axis=0)

pool = ThreadPool(processes = 16)  
block_sums = pool.map(process_block, range(num_blocks))
sum_vector = np.sum(block_sums, axis=0)
mean_vector = sum_vector / num_vectors

print(mean_vector[:10])

[ 0.00097561 -0.033926    0.04665515 -0.08744798 -0.00971177  0.00551533
  0.02714543  0.03753719 -0.02351383 -0.01104514]
Wall time: 1.36 s
